In [1]:
from keras.models import *
import numpy as np
# import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import time

def millis():
    return int(round(time.time() * 1000))

Using TensorFlow backend.
/home/buikhoi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/buikhoi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/buikhoi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/buikhoi/.local/lib/python3.6/site-packages/tensorflow/py

In [2]:
from Models.refinenet.refinennet_model import *
model = Refinenet()
refine_model = model.chose_model(0)[0]
refine_model.load_weights('./Checkpoints/refine30.h5')
# refine_model.summary()

In [3]:
from Models.enet.model import *
#with open('./Checkpoints/enet_nhatrang.json', 'r') as read_file:
 #   run_model = model_from_json(read_file.read())
enet_model = enet_build(6, 320, 240)
enet_model.load_weights('./Checkpoints/enet_rviou.h5')
# enet_model.summary()

In [4]:
with open('./Checkpoints/fcn25.json', 'r') as read_file:
    fcn25_model = model_from_json(read_file.read())
fcn25_model.load_weights('./Checkpoints/fcn25.h5')
# run_model.summary()

In [5]:
def get_prediction(img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = (img/255).astype('float')
    pred = model.predict(np.expand_dims(img, 0))[0]
    temp = np.zeros((240, 320, 3), np.uint8)
    pred[:, :, 0] -= pred[:, :, 1]
    pred[:, :, 0] = np.clip(pred[:, :, 0], 0, 1)
    temp[:, :, 0] = pred[:, :, 0] * 255
    temp[:, :, 1] = pred[:, :, 1] * 255
    temp[:, :, 2] = pred[:, :, 2] * 255
    return temp

In [6]:
cap = cv2.VideoCapture('./Data/SPKT_LHU/Videos/spkt2.avi')
vid_out = cv2.VideoWriter('spkt_compare.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 5, (640, 480))
count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if count < 10:
        count += 1
        continue
    else:
        count = 0
    if ret:
        frame = frame.astype(np.uint8)
        frame = cv2.resize(frame, (320, 240))
        blk_img = np.zeros((480, 640, 3), np.uint8)
        blk_img[:240, :320, :] += frame
        
        fcn25_img = get_prediction(frame, fcn25_model)
        fcn25_img = cv2.putText(fcn25_img, 'FCN25', (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        blk_img[240:480, :320, :] += fcn25_img.astype(np.uint8)
        
        enet_img = get_prediction(frame, enet_model)
        enet_img = cv2.putText(enet_img, 'ENET', (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        blk_img[:240, 320:640, :] += enet_img.astype(np.uint8)
        
        refine_img = get_prediction(frame, refine_model)
        refine_img = cv2.putText(refine_img, 'REFINE', (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        blk_img[240:480, 320:640, :] += refine_img.astype(np.uint8)
        
        cv2.imshow('img', blk_img)
        vid_out.write(blk_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        time.sleep(0.005)
    else:
        break
cap.release()
vid_out.release()
cv2.destroyAllWindows()